# Set Up the Enviroment

In [ ]:
!pip install -U sentence-transformers rank_bm25
!pip install sentence_transformers

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from google.colab import drive
import gzip
import string
import json
import numpy as np
import os
import sys
import re
import math
import pandas as pd
import torch
from pathlib import Path

### Locate the files

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
current_path = "/content/drive/Shareddrives/CIS519 Project/Data"
metadata = pd.read_csv(os.path.join(current_path, 'metadata.csv'))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
os.environ['KAGGLE_USERNAME'] = "shuyanyu" # username from the json file
os.environ['KAGGLE_KEY'] = "49cf99c64a59bf84c51fa76d3d8631b5" # key from the json file

# Data Preprocessing

In [ ]:
metadata.head(3)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [ ]:
list_file = []
list_file.append(metadata[metadata['title'] == 'What are the risks of COVID-19 infection in pregnant women?']['sha'].to_list()[0])
list_file.append(metadata[metadata['title'] == 'What Should Gastroenterologists and Patients Know About COVID-19?']['sha'].to_list()[0])
list_file.append(metadata[metadata['title'] == 'Are high-performing health systems resilient against the COVID-19 epidemic?']['sha'].to_list()[2])

In [ ]:
metadata[metadata['title'] == 'Are high-performing health systems resilient against the COVID-19 epidemic?']

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
404331,hg6n4hun,NaN,WHO,Are high-performing health systems resilient a...,NaN,NaN,NaN,unk,NaN,2020,"Legido-Quigley, Helena; Asgari, Nima; Teo, Yik...",Lancet,NaN,"##BR#""5422",NaN,NaN,NaN,NaN,212594933.0
404332,hg6n4hun,NaN,WHO,Are high-performing health systems resilient a...,NaN,NaN,NaN,unk,NaN,2020,"Legido-Quigley, Helena; Asgari, Nima; Teo, Yik...",Lancet,NaN,#5422,NaN,NaN,NaN,NaN,212594933.0
714952,dycx8i7h,2d0d272a793b99ef896547024c51f486e9552e5a,Elsevier; Medline; PMC,Are high-performing health systems resilient a...,10.1016/s0140-6736(20)30551-1,PMC7124523,32151326.0,no-cc,NaN,2020-03-06,"Legido-Quigley, Helena; Asgari, Nima; Teo, Yik...",Lancet,NaN,NaN,NaN,document_parses/pdf_json/2d0d272a793b99ef89654...,document_parses/pmc_json/PMC7124523.xml.json,https://doi.org/10.1016/s0140-6736(20)30551-1;...,212594933.0


In [ ]:
list_file

['9275fca658701f7e6af4f0514ca2a95f3fd2c5cd',
 'd89abe12ddeaf326dcd6f0af74adc5d8d0d8057d',
 '2d0d272a793b99ef896547024c51f486e9552e5a']

In [ ]:
file_path = os.path.join(current_path, 'baseline_ans.json')

query = []
answers = []
# Parse all the queries and answers from the baseline file
with open(file_path, "r") as read_file:
    data = json.load(read_file)
    count = 1
    for cat in data['categories']:
        for subcat in cat['sub_categories']:
            query.append([count, subcat['nq_name']])
            for answer in subcat['answers']:
                answers.append([count, answer['id']])
            count += 1

output_file_1 = f'queries.txt'
with open(output_file_1, 'w') as output:
    for q in query:
        line = str(q[0]) + ' ' + str(q[1]) + '\n'
        output.write(line)

output_file_2 = f'answers.txt'
with open(output_file_2, 'w') as output:
    for ans in answers:
        candidates = metadata[metadata['cord_uid']==ans[1]]['sha'].to_list()
        if len(candidates) != 0:
            multi_files = candidates[0].split('; ')
            for each in multi_files:
                line = str(ans[0]) + ' ' + str(each) + '\n'
            output.write(line)

# Because our candidiates is randomly drawn from the large pool
# It is possible that those answers are not included
# Save the baseline answers and add to the candidates
known_answers = []
with open('answers.txt', "r") as read_file:
    all_lines = read_file.readlines()
    for each in all_lines:
        data = each.rstrip('\n').split(' ')
        known_answers.append(data[1]+ ".json")

# Document Retrieved from pool

## no tokenization

In [ ]:
stop_words_set = set()
document_count = 0
document_length = {}

def handle_stopwords(current_path):
    stopwords_file = 'words.txt'
    stopwords_path = os.path.join(current_path, stopwords_file)
    stopwords_content = open(stopwords_path, 'r', encoding="ISO-8859-1")
    stopwords_lines = stopwords_content.readlines()
    for temp_line in stopwords_lines:
        temp_line = temp_line.rstrip('\n').split(' ')
        for stopword in temp_line:
            if len(stopword) != 0:
                stop_words_set.add(stopword)

def removeStopwords(tokens):
    temp = 0
    while temp < len(tokens):
        if tokens[temp] in stop_words_set:
            tokens[temp] = ''
        temp += 1
    return tokens

def indexDocument(data_dict, document_scheme, query_scheme, inverted_index):
    lines = []
    doc_id = data_dict['paper_id'] # by observation
    lines.append(data_dict['metadata']['title'])
    if len(data_dict['abstract']) != 0:
        for abstract in data_dict['abstract']:
            lines.append(abstract['text'])
    for text in data_dict['body_text']:
        lines.append(text['text'])
    document_words = []
    
    for line in lines:
        tokens = line.split()
        tokens = removeStopwords(tokens)
        for token in tokens:
            if len(token) != 0:
                document_words.append(str(token))
    # build a temporary dictionary of words and sort
    word_dictionary = {} # word: frequency in document
    for word in document_words:
        if word in word_dictionary:
            word_dictionary[word] += 1
        else:
            word_dictionary[word] = 1
    sorted_wordfrequency = sorted(word_dictionary.items(), key=lambda x: -x[1])
    max_frequency = sorted_wordfrequency[0][1]

    for items in word_dictionary:
        if document_scheme == "tfidf":
            # calculate term frequency for tf = f / max{f}
            word_dictionary[items] = word_dictionary[items] / max_frequency
        if document_scheme == "nxx":
            word_dictionary[items] = 0.5 + (0.5 * word_dictionary[items]) / max_frequency
        
    for word in document_words:
        temp = (doc_id, word_dictionary[word])
        if word in inverted_index:
            inverted_index[word].append(temp)
        else:
            inverted_index[word] = [temp]

In [ ]:
def retrieveDocument(query_string, inverted_index, document_scheme, query_scheme):
    temps = query_string.split(' ')
    query_id = temps[0]
    temp_query_string = ""
    for i in range(1, len(temps)):
        temp_query_string += temps[i] + " "

    # preprocess query string
    query_words = []
    cleaned_query = temp_query_string
    query_tokens = cleaned_query.split()
    query_tokens = removeStopwords(query_tokens)
    for token in query_tokens:
        if len(token) != 0:
            query_words.append(str(token))

    # build query word dictionary
    query_word_dict = {} # in the format of {query words: term frequency}
    for query_word in query_words:
        if query_word in query_word_dict:
            query_word_dict[query_word] += 1
        else:
            query_word_dict[query_word] = 1
    sorted_queryfrequency = sorted(query_word_dict.items(), key=lambda x: -x[1])
    max_queryfrequency = sorted_queryfrequency[0][1]

    # calculate query length
    query_length = 0
    for query_word in query_words:
        if query_scheme == "tfidf" and query_word in inverted_index:
            query_tf = query_word_dict[query_word] / max_queryfrequency # normalize
            query_idf = math.log10(document_count / len(inverted_index[query_word]))
            query_length += math.pow(query_tf * query_idf, 2)
        if query_scheme == "nfx" and query_word in inverted_index:
            query_tf = 0.5 + (0.5 * query_word_dict[query_word]) / max_queryfrequency # normalize
            query_idf = math.log10(document_count / len(inverted_index[query_word]))
            query_length += math.pow(query_tf * query_idf, 2)

    # construct similarity dictionary for this query
    similarity_scores = {} # in the format of {docIDs: similarity scores}
    for query_word in query_words:
        if query_word in inverted_index:
            docId_termfrequency_list = inverted_index[query_word]
            for docID, tf in docId_termfrequency_list:
                if docID in similarity_scores:
                    if query_scheme == "tfidf":
                        query_tf = query_word_dict[query_word] / max_queryfrequency
                        idf = math.log10(document_count / len(inverted_index[query_word]))
                        similarity_scores[docID] += query_tf * idf * tf * idf
                    if query_scheme == "nfx":
                        query_tf = 0.5 + (0.5 * query_word_dict[query_word]) / max_queryfrequency
                        idf = math.log10(document_count / len(inverted_index[query_word]))
                        similarity_scores[docID] += query_tf * idf * tf
                else:
                    if query_scheme == "tfidf":
                        query_tf = query_word_dict[query_word] / max_queryfrequency
                        idf = math.log10(document_count / len(inverted_index[query_word]))
                        similarity_scores[docID] = query_tf * idf * tf * idf
                    if query_scheme == "nfx":
                        query_tf = 0.5 + (0.5 * query_word_dict[query_word]) / max_queryfrequency
                        idf = math.log10(document_count / len(inverted_index[query_word]))
                        similarity_scores[docID] = query_tf * idf * tf
                        
    # finalize calculating the similarity_scores
    for item in similarity_scores:
        temp_length_mul = math.sqrt(document_length[item]) * math.sqrt(query_length)
        if temp_length_mul != 0:
            similarity_scores[item] = similarity_scores[item] / temp_length_mul
    
    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: -x[1])
    return sorted_similarity

In [ ]:
if __name__ == "__main__":
    current_path = "/content/drive/Shareddrives/CIS519 Project/Data"
    document_scheme = "tfidf"
    query_scheme = "tfidf"
    folder_addr = "sample_train_set"
    queries = "queries.txt"

    folder_path = os.path.join(current_path, folder_addr)
    files = os.listdir(folder_path)
    files.sort()
    inverted_index = {}
    # in the format of: {word: [(DocId, term_frequency), ...]}

    total_words = []
    handle_stopwords(current_path)

    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, "r") as read_file:
          data = json.load(read_file)
          indexDocument(data, document_scheme, query_scheme, inverted_index)
          document_count += 1
    
    # complete building invrted_index for fast access
    # notice that the lenth of the list is the document frequency
    # now calculate the document length for similarity
    for item in inverted_index:
        list_docID_tf = inverted_index[item]
        item_document_frequency = len(list_docID_tf)
        for docID_tf in list_docID_tf:
            if docID_tf[0] in document_length:
                if document_scheme == "tfidf":
                    idf = math.log10(document_count/item_document_frequency)
                    document_length[docID_tf[0]] += math.pow(docID_tf[1]*idf, 2)
                if document_scheme == "nxx":
                    document_length[docID_tf[0]] += math.pow(docID_tf[1], 2)
            else:
                if document_scheme == "tfidf":
                    idf = math.log10(document_count/item_document_frequency)
                    document_length[docID_tf[0]] = math.pow(docID_tf[1]*idf, 2)
                if document_scheme == "nxx":
                    document_length[docID_tf[0]] = math.pow(docID_tf[1], 2)

    query_path = os.path.join("", queries)
    query_content = open(query_path, 'r')
    query_lines = query_content.readlines()
    list_similarity_scores = []
    for query in query_lines:
        similarity_scores = retrieveDocument(query, inverted_index, document_scheme, query_scheme)
        list_similarity_scores.append(similarity_scores)
    output_file = f'sample_test_notokenize.{document_scheme}.{query_scheme}.output'
    query_count = 1
    with open(output_file, 'w') as output:
        for similarity in list_similarity_scores:
            for item in similarity:
                line = str(query_count) + ' ' + str(item[0]) + ' ' + str(item[1]) + '\n'
                output.write(line)
            query_count += 1

## Tokenization

### Method 1

In [ ]:
stop_words_set = set()
document_count = 0
document_length = {}

def handle_stopwords(current_path):
    stopwords_file = 'words.txt'
    stopwords_path = os.path.join(current_path, stopwords_file)
    stopwords_content = open(stopwords_path, 'r', encoding="ISO-8859-1")
    stopwords_lines = stopwords_content.readlines()
    for temp_line in stopwords_lines:
        temp_line = temp_line.rstrip('\n').split(' ')
        for stopword in temp_line:
            if len(stopword) != 0:
                stop_words_set.add(stopword)

def removeSGML(input_line):
    cleaned_line = re.sub("<.*?>", "", input_line)
    return cleaned_line

def tokenizeText(in_str):
    months = ('Jan.', 'January', 'Feb.', 'February', 'Mar.', 'March', 
            'Apr.', 'April', 'May', 'Jun.', 'June', 'Jul.', 'July', 
            'Aug.', 'August', 'Sept.', 'September', 'Oct.', 'October', 
            'Nov.', 'November', 'Dec.', 'December')
    wh_words = ('it', 'she', 'he', 'who', 'why', 'when', 'what', 'where',
              'how', 'there', 'that')
    
    # split in_str by whitespace
    untoken = in_str.split()
    tokens = []
    skip = 0

    for i in range(len(untoken)):
        # skip word that is part of token
        if 0 < skip:
            skip -= 1
            continue

        # Tokenization of dates (keep dates together)
        if untoken[i] in months:
            if i < len(untoken) - 1:
                rest = None
                # (Mar./March 12, 1991)
                tok = ' '
                if i < len(untoken) - 2:
                    if re.match(r'\d\d,$', untoken[i+1]) and re.match(r'\d\d\d\d', untoken[i+2]):
                        year, rest = untoken[i+2][:4], untoken[i+2][4:]
                        tok = tok.join((untoken[i], untoken[i+1], year))
                        skip += 2
                # (Mar./March 12)
                elif re.match(r'\d\d[\W]*$', untoken[i+1]):
                    date, rest = untoken[i+1][:2], untoken[i+1][2:]
                    tok = tok.join((untoken[i], date))
                    skip += 1
                # (Mar./March 1991)
                elif re.match(r'\d\d\d\d', untoken[i+1]):
                    year, rest = untoken[i+1][:4], untoken[i+1][4:]
                    tok = tok.join((untoken[i], year))
                    skip += 1
                tokens.append(tok)
                if rest:
                    tokens.append(rest)
                continue

        # (3/12/91)
        if re.match(r'\d{1,4}/\d{1,4}/\d{1,4}', untoken[i]):
            if not untoken[i][-1].isdigit():
                tok, rest = untoken[i][:-1], untoken[i][-1]
                tokens.append(tok)
                tokens.append(rest)
            else:
                tokens.append(untoken[i])
            continue

        # Do not tokenize acronyms, abbreviations, numbers
        # (55 B.C.)
        if untoken[i].isdigit():
            if i < len(untoken) - 1:
                if untoken[i+1][0].isupper() and (untoken[i+1].count('.') > 1):
                    if not untoken[i+1][-1].isalpha() and untoken[i+1][-1] != '.':
                        tok = ' '.join((untoken[i], untoken[i+1][:-1]))
                        rest = untoken[i+1][-1]
                        tokens.append(tok)
                        tokens.append(rest)
                    else:
                        tok = ' '.join((untoken[i], untoken[i+1]))
                        tokens.append(tok)
                    skip += 1
                    continue
        # (Mr.)(U.S.A.)
        if untoken[i][0].isupper() and (untoken[i].count('.') >= 1):
            if not untoken[i].endswith('.'):
                tok, rest = untoken[i][:-1], untoken[i][-1]
                tokens.append(tok)
                tokens.append(rest)
            else:
                tokens.append(untoken[i])
            continue

        # Tokenize of quotation
        if untoken[i].startswith('\'') or untoken[i].startswith('‘'):
            tok, rest = untoken[i][1:], untoken[i][:1]
            tokens.append(rest)
            tokens.append(tok)
            continue
        if untoken[i].endswith('\'') or untoken[i].endswith('’'):
            tok, rest = untoken[i][:-1], untoken[i][-1]
            tokens.append(tok)
            tokens.append(rest)
            continue

        # Tokenize of contraction, possessive
        if untoken[i].endswith('\'s') or untoken[i].endswith('’s'):
            tok = untoken[i][:-2]
            rest = 'is' if tok in wh_words else untoken[i][-2:]
            tokens.append(tok)
            tokens.append(rest)
            continue
        if untoken[i].count('\'') == 1 or untoken[i].count('’') == 1:
            tok, rest = None, None
            if untoken[i].endswith('\'m') or untoken[i].endswith('’m'):
                tok = untoken[i][:-2]
                rest = 'am'
            elif untoken[i].endswith('\'d') or untoken[i].endswith('’d'):
                tok = untoken[i][:-2]
                rest = 'would'
            elif untoken[i].endswith('\'re') or untoken[i].endswith('’re'):
                tok = untoken[i][:-3]
                rest = 'are'
            elif untoken[i].endswith('\'ve') or untoken[i].endswith('’ve'):
                tok = untoken[i][:-3]
                rest = 'have'
            elif untoken[i].endswith('\'ll') or untoken[i].endswith('’ll'):
                tok = untoken[i][:-3]
                rest = 'will'
            elif untoken[i].endswith('n\'t') or untoken[i].endswith('n’t'):
                tok = untoken[i][:-3]
                rest = 'not'
            if tok and rest:
                tokens.append(tok)
                tokens.append(rest)
                continue

        # Tokenization of - (keep phrases separated by - together)
        if untoken[i].count('-') >= 1:
            if untoken[i][-1] == '-':
                tok = untoken[i]
                tokens.append(tok)
                continue
            if re.match('[\W_]', untoken[i][-1]):
                tok, rest = untoken[i][:-1], untoken[i][-1]
                tokens.append(tok)
                tokens.append(rest)
                continue
            else:
                tok = untoken[i]
                tokens.append(tok)
                continue

        # Do not tokenize numbers
        if untoken[i][0].isdigit():
            tmp_tok = re.sub(r'[\W_]', '', untoken[i])
            if tmp_tok.isdigit():
                if not untoken[i][-1].isdigit():
                    tok, rest = untoken[i][:-1], untoken[i][-1]
                    tokens.append(tok)
                    tokens.append(rest)
                else:
                    tok = untoken[i]
                    tokens.append(tok)
                continue

        # Normal
        if untoken[i].isalnum():
            tokens.append(untoken[i])
        else:
            for tok in re.split('(\W)', untoken[i]):
                if tok:
                    tokens.append(tok)
    return tokens

def removeStopwords(tokens):
    temp = 0
    while temp < len(tokens):
        if tokens[temp] in stop_words_set:
            tokens[temp] = ''
        temp += 1
    return tokens

def indexDocument(data_dict, document_scheme, query_scheme, inverted_index):
    lines = []
    doc_id = data_dict['paper_id'] # by observation
    lines.append(data_dict['metadata']['title'])
    if len(data_dict['abstract']) != 0:
        for abstract in data_dict['abstract']:
            lines.append(abstract['text'])
    for text in data_dict['body_text']:
        lines.append(text['text'])
    document_words = []
    
    for line in lines:
        cleaned_line = removeSGML(line)
        tokens = tokenizeText(cleaned_line)
        tokens = removeStopwords(tokens)
        for token in tokens:
            if len(token) != 0:
                document_words.append(str(token))
    # build a temporary dictionary of words and sort
    word_dictionary = {} # word: frequency in document
    for word in document_words:
        if word in word_dictionary:
            word_dictionary[word] += 1
        else:
            word_dictionary[word] = 1
    sorted_wordfrequency = sorted(word_dictionary.items(), key=lambda x: -x[1])
    max_frequency = sorted_wordfrequency[0][1]

    for items in word_dictionary:
        if document_scheme == "tfidf":
            # calculate term frequency for tf = f / max{f}
            word_dictionary[items] = word_dictionary[items] / max_frequency
        if document_scheme == "nxx":
            word_dictionary[items] = 0.5 + (0.5 * word_dictionary[items]) / max_frequency
        
    for word in document_words:
        temp = (doc_id, word_dictionary[word])
        if word in inverted_index:
            inverted_index[word].append(temp)
        else:
            inverted_index[word] = [temp]

In [ ]:
def retrieveDocument(query_string, inverted_index, document_scheme, query_scheme):
    temps = query_string.split(' ')
    query_id = temps[0]
    temp_query_string = ""
    for i in range(1, len(temps)):
        temp_query_string += temps[i] + " "

    # preprocess query string
    query_words = []
    cleaned_query = removeSGML(temp_query_string)
    query_tokens = tokenizeText(cleaned_query)
    query_tokens = removeStopwords(query_tokens)
    for token in query_tokens:
        if len(token) != 0:
            query_words.append(str(token))

    # build query word dictionary
    query_word_dict = {} # in the format of {query words: term frequency}
    for query_word in query_words:
        if query_word in query_word_dict:
            query_word_dict[query_word] += 1
        else:
            query_word_dict[query_word] = 1
    sorted_queryfrequency = sorted(query_word_dict.items(), key=lambda x: -x[1])
    max_queryfrequency = sorted_queryfrequency[0][1]

    # calculate query length
    query_length = 0
    for query_word in query_words:
        if query_scheme == "tfidf" and query_word in inverted_index:
            query_tf = query_word_dict[query_word] / max_queryfrequency # normalize
            query_idf = math.log10(document_count / len(inverted_index[query_word]))
            query_length += math.pow(query_tf * query_idf, 2)
        if query_scheme == "nfx" and query_word in inverted_index:
            query_tf = 0.5 + (0.5 * query_word_dict[query_word]) / max_queryfrequency # normalize
            query_idf = math.log10(document_count / len(inverted_index[query_word]))
            query_length += math.pow(query_tf * query_idf, 2)

    # construct similarity dictionary for this query
    similarity_scores = {} # in the format of {docIDs: similarity scores}
    for query_word in query_words:
        if query_word in inverted_index:
            docId_termfrequency_list = inverted_index[query_word]
            for docID, tf in docId_termfrequency_list:
                if docID in similarity_scores:
                    if query_scheme == "tfidf":
                        query_tf = query_word_dict[query_word] / max_queryfrequency
                        idf = math.log10(document_count / len(inverted_index[query_word]))
                        similarity_scores[docID] += query_tf * idf * tf * idf
                    if query_scheme == "nfx":
                        query_tf = 0.5 + (0.5 * query_word_dict[query_word]) / max_queryfrequency
                        idf = math.log10(document_count / len(inverted_index[query_word]))
                        similarity_scores[docID] += query_tf * idf * tf
                else:
                    if query_scheme == "tfidf":
                        query_tf = query_word_dict[query_word] / max_queryfrequency
                        idf = math.log10(document_count / len(inverted_index[query_word]))
                        similarity_scores[docID] = query_tf * idf * tf * idf
                    if query_scheme == "nfx":
                        query_tf = 0.5 + (0.5 * query_word_dict[query_word]) / max_queryfrequency
                        idf = math.log10(document_count / len(inverted_index[query_word]))
                        similarity_scores[docID] = query_tf * idf * tf

    # finalize calculating the similarity_scores
    for item in similarity_scores:
        temp_length_mul = math.sqrt(document_length[item]) * math.sqrt(query_length)
        if temp_length_mul != 0:
            similarity_scores[item] = similarity_scores[item] / temp_length_mul
    
    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: -x[1])
    return sorted_similarity

### Method 2

In [ ]:
stop_words_set = set()
document_count = 0
document_length = {}

def handle_stopwords(current_path):
    current_path = "/content/drive/Shareddrives/CIS519 Project/Data"
    stopwords_file = 'words.txt'
    stopwords_path = os.path.join(current_path, stopwords_file)
    stopwords_content = open(stopwords_path, 'r', encoding="ISO-8859-1")
    stopwords_lines = stopwords_content.readlines()
    for temp_line in stopwords_lines:
        temp_line = temp_line.rstrip('\n')
        temp_line = temp_line.split(' ')
        for stopword in temp_line:
            if len(stopword) != 0:
                stop_words_set.add(stopword)

def removeSGML(input_line):
    cleaned_line = re.sub("<.*?>", "", input_line)
    return cleaned_line

def tokenizeText(cleaned_line):
    data_month = ['january', 'jan', 'jan.', 'february', 'feb', 'feb.',
                  'march', 'mar', 'mar.', 'april', 'apr', 'apr.',
                  'may', 'may.', 'june', 'jun', 'jun.', 
                  'july', 'jul', 'jul.', 'august', 'aug', 'aug.',
                  'septemer', 'sept', 'sept.', 'october', 'oct', 'oct.',
                  'november', 'nov', 'nov.', 'december', 'dec', 'dec.']
    weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    common_contractions = {
        "'t": " not",
        "'m": " am",
        "'s": " is",
        "'re": " are",
        "'ve": " have",
        "'ll": " will",
        "'d": " would",
        "'cause": " because"
    }
    tokens = []
    temp_tokens = cleaned_line.split(" ")
    # by observing the commond patterns in reg, hard code the case
    count = 0
    while count < len(temp_tokens):
        temp_word = temp_tokens[count]
        temp_word = re.sub("\n", '', temp_word) # clean the \n

        # first if no signs existed
        if ('.' not in temp_word) and (',' not in temp_word):
            if ('(' not in temp_word) and (')' not in temp_word):
                if ('-' not in temp_word) and ('\'' not in temp_word):
                    tokens.append(temp_word)
                    count += 1
                    continue

        final_token = temp_word
        # tokenization of . (do not tokenize acronyms, abbreviations, numbers)
        if '.' in final_token:
            if len(final_token) == 1:
                final_token = ''
            elif final_token.endswith('.'): # case: date (e.g. Mar. 12)
                if final_token[:-1].lower() in data_month:
                  if count + 1 < len(temp_tokens):
                    if temp_tokens[count + 1].isdigit():
                        final_token = final_token[:-1] + ' ' + temp_tokens[count + 1]
                        count += 2 # jump to the next possible token
                        tokens.append(final_token)
                        continue
                    else:
                        final_token = final_token[:-1]
            elif final_token[0].isdigit(): # case: numbers
                if final_token.endswith('.'):
                    final_token = final_token[:-1]
                else:
                    final_token = final_token
            elif final_token[0].isalpha() and final_token[0].isupper(): # case: acronyms or abbreviations
                final_token = final_token.lower()

        #tokenization of ' (expand when needed, e.g., I’m -> I am; tokenize the possessive, e.g., Sunday’s -> Sunday ‘s; etc.)
        if '\'' in final_token:
            if final_token.startswith('\''):
                if final_token.endswith('\''): # case: remove quotations
                    final_token = final_token[1:-1].lower()
                else:
                    final_token = final_token[1:].lower()
            elif final_token.endswith('\''):
                final_token = final_token[:-1].lower()
            else: # case: when ' is in the middle
                k = 0
                while k < len(final_token):
                    if final_token[k] == '\'':
                        break
                    k += 1
                # case: Sunday's
                if final_token[:k].lower() in weekdays:
                    final_token = final_token[:k] + ' \'s'
                elif final_token[k:k+2] in common_contractions.keys(): # case: expand
                    final_token = final_token[:k] + common_contractions[final_token[k:k+2]] + final_token[k+2:]
                elif final_token[k:k+3] in common_contractions.keys():
                    final_token = final_token[:k] + common_contractions[final_token[k:k+3]] + final_token[k+3:]
        # tokenization of - (keep phrases separated by - together)
        if '-' in final_token:
            if final_token.startswith('-'):
                if final_token.endswith('-'): # case: remove quotations
                    final_token = final_token[1:-1].lower()
                else:
                    final_token = final_token[1:].lower()
            elif final_token.endswith('-'):
                final_token = final_token[:-1].lower()
            else: # case: when - is in the middle
                final_token = final_token

        # tokenization of , (do not tokenize numbers)
        if ',' in final_token:
            if final_token.startswith(','):
                final_token = final_token[1:].lower()
            elif final_token.endswith(','):
                if final_token[:-1].lower() in data_month:
                  if count + 1 < len(temp_tokens):
                    if temp_tokens[count + 1].isdigit():
                        final_token = final_token[:-1] + ' ' + temp_tokens[count + 1]
                        count += 2 # jump to the next possible token
                        tokens.append(final_token)
                        continue
                final_token = final_token[:-1].lower()
            else: # case: when , is in the middle
                if final_token[0].isdigit(): # case: number
                    final_token = final_token

        if '(' in final_token:
            if final_token.startswith('('):
                final_token = final_token[1:].lower()
            if final_token.endswith('('):
                final_token = final_token[:-1].lower()
        if ')' in final_token:
            if final_token.startswith(')'):
                final_token = final_token[1:].lower()
            if final_token.endswith(')'):
                final_token = final_token[:-1].lower()

        tokens.append(final_token.lower())
        count += 1
    return tokens

def removeStopwords(tokens):
    temp = 0
    while temp < len(tokens):
        if tokens[temp] in stop_words_set:
            tokens[temp] = ''
        temp += 1
    return tokens

def indexDocument(data_dict, document_scheme, query_scheme, inverted_index):
    #lines = document_string.split('\n')
    lines = []
    doc_id = data_dict['paper_id'] # by observation
    lines.append(data_dict['metadata']['title'])
    if len(data_dict['abstract']) != 0:
        for abstract in data_dict['abstract']:
            lines.append(abstract['text'])
    for text in data_dict['body_text']:
        lines.append(text['text'])
    document_words = []
    
    for line in lines:
        cleaned_line = removeSGML(line)
        tokens = tokenizeText(cleaned_line)
        tokens = removeStopwords(tokens)
        for token in tokens:
            if len(token) != 0:
                document_words.append(str(token))

    # build a temporary dictionary of words and sort
    word_dictionary = {} # word: frequency in document
    for word in document_words:
        if word in word_dictionary:
            word_dictionary[word] += 1
        else:
            word_dictionary[word] = 1
    sorted_wordfrequency = sorted(word_dictionary.items(), key=lambda x: -x[1])
    max_frequency = sorted_wordfrequency[0][1]

    for items in word_dictionary:
        if document_scheme == "tfidf":
            # calculate term frequency for tf = f / max{f}
            word_dictionary[items] = word_dictionary[items] / max_frequency
        if document_scheme == "nxx":
            word_dictionary[items] = 0.5 + (0.5 * word_dictionary[items]) / max_frequency
        
    
    for word in document_words:
        temp = (doc_id, word_dictionary[word])
        if word in inverted_index:
            inverted_index[word].append(temp)
        else:
            inverted_index[word] = [temp]

In [ ]:
def retrieveDocument(query_string, inverted_index, document_scheme, query_scheme):
    temps = query_string.split(' ')
    query_id = temps[0]
    temp_query_string = ""
    for i in range(1, len(temps)):
        temp_query_string += temps[i] + " "

    # preprocess query string
    query_words = []
    cleaned_query = removeSGML(temp_query_string)
    query_tokens = tokenizeText(cleaned_query)
    query_tokens = removeStopwords(query_tokens)
    for token in query_tokens:
        if len(token) != 0:
            query_words.append(str(token))

    # build query word dictionary
    query_word_dict = {} # in the format of {query words: term frequency}
    for query_word in query_words:
        if query_word in query_word_dict:
            query_word_dict[query_word] += 1
        else:
            query_word_dict[query_word] = 1
    sorted_queryfrequency = sorted(query_word_dict.items(), key=lambda x: -x[1])
    max_queryfrequency = sorted_queryfrequency[0][1]

    # calculate query length
    query_length = 0
    for query_word in query_words:
        if query_scheme == "tfidf" and query_word in inverted_index:
            query_tf = query_word_dict[query_word] / max_queryfrequency # normalize
            query_idf = math.log10(document_count / len(inverted_index[query_word]))
            query_length += math.pow(query_tf * query_idf, 2)
        if query_scheme == "nfx" and query_word in inverted_index:
            query_tf = 0.5 + (0.5 * query_word_dict[query_word]) / max_queryfrequency # normalize
            query_idf = math.log10(document_count / len(inverted_index[query_word]))
            query_length += math.pow(query_tf * query_idf, 2)

    # construct similarity dictionary for this query
    similarity_scores = {} # in the format of {docIDs: similarity scores}
    for query_word in query_words:
        if query_word in inverted_index:
            docId_termfrequency_list = inverted_index[query_word]
            for docID, tf in docId_termfrequency_list:
                if docID in similarity_scores:
                    if query_scheme == "tfidf":
                        query_tf = query_word_dict[query_word] / max_queryfrequency
                        idf = math.log10(document_count / len(inverted_index[query_word]))
                        similarity_scores[docID] += query_tf * idf * tf * idf
                    if query_scheme == "nfx":
                        query_tf = 0.5 + (0.5 * query_word_dict[query_word]) / max_queryfrequency
                        idf = math.log10(document_count / len(inverted_index[query_word]))
                        similarity_scores[docID] += query_tf * idf * tf
                else:
                    if query_scheme == "tfidf":
                        query_tf = query_word_dict[query_word] / max_queryfrequency
                        idf = math.log10(document_count / len(inverted_index[query_word]))
                        similarity_scores[docID] = query_tf * idf * tf * idf
                    if query_scheme == "nfx":
                        query_tf = 0.5 + (0.5 * query_word_dict[query_word]) / max_queryfrequency
                        idf = math.log10(document_count / len(inverted_index[query_word]))
                        similarity_scores[docID] = query_tf * idf * tf

    # finalize calculating the similarity_scores
    for item in similarity_scores:
        temp_length_mul = math.sqrt(document_length[item]) * math.sqrt(query_length)
        if temp_length_mul != 0:
            similarity_scores[item] = similarity_scores[item] / temp_length_mul
    
    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: -x[1])
    return sorted_similarity

# Load Files

In [ ]:
current_path = "/content/drive/Shareddrives/CIS519 Project/Data"
document_scheme = "tfidf"
query_scheme = "tfidf"
folder_addr = "sample_train_set"
queries = "queries.txt"

folder_path = os.path.join(current_path, folder_addr)
files = os.listdir(folder_path)
files.sort()
inverted_index = {}
# in the format of: {word: [(DocId, term_frequency), ...]}

total_words = []
handle_stopwords(current_path)

document_count = 0
for file_name in files:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, "r") as read_file:
        data = json.load(read_file)
        indexDocument(data, document_scheme, query_scheme, inverted_index)
        document_count += 1

#TF-IDF

In [ ]:
# complete building invrted_index for fast access
# notice that the lenth of the list is the document frequency
# now calculate the document length for similarity
print(document_count)
for item in inverted_index:
    list_docID_tf = inverted_index[item]
    item_document_frequency = len(list_docID_tf)
    for docID_tf in list_docID_tf:
        if docID_tf[0] in document_length:
            if document_scheme == "tfidf":
                idf = math.log10(document_count/item_document_frequency)
                document_length[docID_tf[0]] += math.pow(docID_tf[1]*idf, 2)
            if document_scheme == "nxx":
                document_length[docID_tf[0]] += math.pow(docID_tf[1], 2)
        else:
            if document_scheme == "tfidf":
                idf = math.log10(document_count/item_document_frequency)
                document_length[docID_tf[0]] = math.pow(docID_tf[1]*idf, 2)
            if document_scheme == "nxx":
                document_length[docID_tf[0]] = math.pow(docID_tf[1], 2)

query_path = os.path.join("", queries)
query_content = open(query_path, 'r')
query_lines = query_content.readlines()

list_similarity_scores = []
for query in query_lines:
    similarity_scores = retrieveDocument(query, inverted_index, document_scheme, query_scheme)
    list_similarity_scores.append(similarity_scores)
output_file = f'sample_test_tokenize2.{document_scheme}.{query_scheme}.output'

query_count = 1
with open(output_file, 'w') as output:
    for similarity in list_similarity_scores:
        for item in similarity:
            line = str(query_count) + ' ' + str(item[0]) + '\n'
            output.write(line)
        query_count += 1

10099


# Metrics: Test for Macro Average



In [ ]:
# metrics 
# test for macro average

def marco_avg(output_file):
    reljudge = {}
    with open('answers.txt', 'r') as data:
        for line in data:
            queryid, docid = line.split()
            if queryid not in reljudge:
                reljudge[queryid] = set()
            reljudge[queryid].add(docid)

    tfidf = {}
    with open(output_file, 'r') as data:
        for line in data:
            queryid, docid = line.split()
            if queryid not in tfidf:
                tfidf[queryid] = list()
            tfidf[queryid].append(docid)

    marco_avg_top(reljudge, tfidf, 1)
    marco_avg_top(reljudge, tfidf, 5)
    marco_avg_top(reljudge, tfidf, 10)
    marco_avg_top(reljudge, tfidf, 50)
    marco_avg_top(reljudge, tfidf, 100)
    marco_avg_top(reljudge, tfidf, 500)
    marco_avg_top(reljudge, tfidf, 1000)


def marco_avg_top(reljudge, tfidf, top):
    num_queries = len(reljudge)
    precision = {}
    recall = {}
    total_retrieved_unique = set()
    for queryid in range(1, num_queries+1):
        qid = str(queryid)
        top_retrieved = set()
        for i in range(top):
            if queryid <= 27 and i < len(tfidf[qid]): 
                top_retrieved.add(tfidf[qid][i])
                total_retrieved_unique.add(tfidf[qid][i])
        rel_returned = top_retrieved.intersection(reljudge[qid])
        precision[queryid] = len(rel_returned)
        recall[queryid] = len(rel_returned) / len(reljudge[qid])

    total_rels = 0
    for queryid, num_rel in precision.items():
        total_rels += num_rel

    precise_score = (total_rels / top) / num_queries
    print(f'Top {top}')
    print(f'  retrieved(relevant) = {total_rels}')
    total_retrieved = top * num_queries
    print(f'  retrieved(total) = {total_retrieved}')
    print('  precision(macro-avg) = ' + '{:.2%}'.format(precise_score))

    total_recall_score = 0
    for qid, recall_score in recall.items():
        total_recall_score += recall_score

    total_recall_score_norm = total_recall_score / num_queries
    print('  recall(macro-avg) = ' + '{:.2%}'.format(total_recall_score_norm))

    print('total ' + str(len(total_retrieved_unique)))

In [ ]:
print('tfidf-tfidf')
marco_avg('sample_test_tokenize2.tfidf.tfidf.output')
#print('\nnxx-nfx')
#marco_avg('cranfield.nxx.nfx.output')

tfidf-tfidf
Top 1
  retrieved(relevant) = 0
  retrieved(total) = 27
  precision(macro-avg) = 0.00%
  recall(macro-avg) = 0.00%
total 7
Top 5
  retrieved(relevant) = 5
  retrieved(total) = 135
  precision(macro-avg) = 3.70%
  recall(macro-avg) = 4.93%
total 38
Top 10
  retrieved(relevant) = 11
  retrieved(total) = 270
  precision(macro-avg) = 4.07%
  recall(macro-avg) = 14.63%
total 72
Top 50
  retrieved(relevant) = 25
  retrieved(total) = 1350
  precision(macro-avg) = 1.85%
  recall(macro-avg) = 20.78%
total 274
Top 100
  retrieved(relevant) = 37
  retrieved(total) = 2700
  precision(macro-avg) = 1.37%
  recall(macro-avg) = 37.13%
total 525
Top 500
  retrieved(relevant) = 56
  retrieved(total) = 13500
  precision(macro-avg) = 0.41%
  recall(macro-avg) = 47.37%
total 2192
Top 1000
  retrieved(relevant) = 65
  retrieved(total) = 27000
  precision(macro-avg) = 0.24%
  recall(macro-avg) = 53.65%
total 3739


# Prepare input data for the following sections
Note: The RAM is not capable to run both BM25 and BERT. Choose only one to run for each Colab Session.

In [ ]:
passages = []
folder_addr = "sample_train_set"
queries = "queries.txt"

folder_path = os.path.join(current_path, folder_addr)
files = os.listdir(folder_path)

for file_name in files:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r') as read_file:
        data = json.load(read_file)
        txt = ""
        id = data['paper_id']
        for each in data['body_text']:
            txt += each['text']
        passages.append((id, txt))
  
print("Passages:", len(passages))

Passages: 10099


# BM25

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from sklearn.feature_extraction import _stop_words
from rank_bm25 import BM25Okapi

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")


#We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/794 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# We encode all passages into our vector space. This takes about 5 minutes (depends on your GPU speed)
corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/316 [00:00<?, ?it/s]

In [ ]:
from tqdm.autonotebook import tqdm

# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)
        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

tokenized_corpus = []
for passage in tqdm(passages):
    text = passage[1]
    tokenized_corpus.append(bm25_tokenizer(text))

  0%|          | 0/10099 [00:00<?, ?it/s]

In [ ]:
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
def search(query, top_k, bi_encoder_flag):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -5)[-5:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    
    print("Top-" + str(top_k) + " lexical search (BM25) hits")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}\t{}".format(hit['score'], passages[hit['corpus_id']][0], passages[hit['corpus_id']][1].replace("\n", " ")))

    ##### Semantic Search #####
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    cross_inp = [[query, passages[hit['corpus_id']][1]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-" + str(top_k) + " Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    id = passages[hit['corpus_id']][0]
    for hit in hits[0:top_k]:
        print("\t{:.3f}\t{}\t{}".format(hit['score'], id, passages[hit['corpus_id']][1].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-" + str(top_k) + " Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    id = passages[hit['corpus_id']][0]
    for hit in hits[0:top_k]:
        print("\t{:.3f}\t{}\t{}".format(hit['score'], id, passages[hit['corpus_id']][1].replace("\n", " ")))

In [ ]:
temp_test = search("What are the risks of COVID-19 infection in pregnant women?", 3, True)

Input question: What are the risks of COVID-19 infection in pregnant women?
Top-3 lexical search (BM25) hits
	23.999	b2696d577d29fb5656f54ad052d6251c2c2ed084	In December 2019, a series of pneumonia cases were initially observed in Wuhan City, Hubei province, China, then a novel coronavirus, severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), was identified as the pathogen for the epidemic pneumonia [1] . On February 12, the World Health Organization (WHO) officially named the infectious disease as coronavirus disease 2019 (COVID-19) [2] . On March 12, 2020 , the COVID-19 viral disease has swept into at least 117 countries and killed more than 4000 people, and WHO officially announced a pandemic of COVID-19 viral disease [3] . As of March 22, 2020 , many studies reported the clinical and immunological characteristics of the general population with COVID-19, but the studies on pregnant women with COVID-19 and their fetuses were relatively rare [4, 5] . More investigations are n

In [ ]:
def search_test_bm25(query, top_k, bi_encoder_flag):

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -5)[-5:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    ##### Semantic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, passages[hit['corpus_id']][1]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)

    # Output of top-5 hits from re-ranker
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    
    query_retrieved_documents = []
    if bi_encoder_flag:
        for hit in hits[0:top_k]:
            query_retrieved_documents.append((hit['score'], passages[hit['corpus_id']][0].replace("\n", " ")))
    else:
        for hit in hits[0:top_k]:
            query_retrieved_documents.append((hit['cross-score'], passages[hit['corpus_id']][0].replace("\n", " ")))
    return query_retrieved_documents

In [ ]:
def test_bm25(top_k):
    query_path = os.path.join("queries.txt")
    query_content = open(query_path, 'r')
    query_lines = query_content.readlines()
    list_similarity_scores = []
    final_output = []
    for query in query_lines:
        query_id = query.split(' ')[0]
        query = query[2:]
        retrieved_list = search_test_bm25(query, top_k, True)
        for score, file_id in retrieved_list:
            output = str(query_id) + ' ' + file_id
            final_output.append(output)
    
    output_file = f'sample_test_bm25.output'
    with open(output_file, 'w') as output:
        for line in final_output:
            line = line + '\n'
            output.write(line)

In [ ]:
test_bm25(500)

In [ ]:
print('BM25')
marco_avg('sample_test_bm25.output')

BM25
Top 1
  retrieved(relevant) = 3
  retrieved(total) = 27
  precision(macro-avg) = 11.11%
  recall(macro-avg) = 4.78%
total 21
Top 5
  retrieved(relevant) = 10
  retrieved(total) = 135
  precision(macro-avg) = 7.41%
  recall(macro-avg) = 8.03%
total 91
Top 10
  retrieved(relevant) = 19
  retrieved(total) = 270
  precision(macro-avg) = 7.04%
  recall(macro-avg) = 13.08%
total 164
Top 50
  retrieved(relevant) = 31
  retrieved(total) = 1350
  precision(macro-avg) = 2.30%
  recall(macro-avg) = 23.32%
total 691
Top 100
  retrieved(relevant) = 35
  retrieved(total) = 2700
  precision(macro-avg) = 1.30%
  recall(macro-avg) = 26.23%
total 1211
Top 500
  retrieved(relevant) = 56
  retrieved(total) = 13500
  precision(macro-avg) = 0.41%
  recall(macro-avg) = 44.99%
total 4021
Top 1000
  retrieved(relevant) = 56
  retrieved(total) = 27000
  precision(macro-avg) = 0.21%
  recall(macro-avg) = 44.99%
total 4021


# Bert Rerank

In [ ]:
import transformers

In [ ]:
def segment_documents(docs, max_doc_length=450):

    segmented_docs = []
    for each in docs:
        doc = each[1]
        id = each[0]
        split_to_words = doc.split(" ")
      
        # If the document is longer than our maximum length, split it up into smaller segments and add them to the list 
        if len(split_to_words) > max_doc_length:
            for doc_segment in range(0, len(split_to_words), max_doc_length):
                segmented_docs.append((id, " ".join(split_to_words[doc_segment:doc_segment + max_doc_length])))
        else:
            segmented_docs.append((id, doc))
            
    return segmented_docs

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_top_k_articles(query, docs, k=2):

    vectorizer = TfidfVectorizer(analyzer="word", stop_words='english')

    txt = []
    for each in docs:
        txt.append(each[1])
    query_and_docs = [query] + txt
    matrix = vectorizer.fit_transform(query_and_docs)

    scores = []

    for i in range(1, len(query_and_docs)):
        scores.append(cosine_similarity(matrix[0], matrix[i])[0][0])

    sorted_list = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
    top_doc_indices = [x[0] for x in sorted_list[:k]]
    top_docs = [docs[x] for x in top_doc_indices]
    
    return top_docs

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
def answer_question(question, answer_text):

    input_ids = tokenizer.encode(question, answer_text, max_length=512)
    
    # ======== Set Segment IDs ========
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    assert len(segment_ids) == len(input_ids)

    outputs = model(torch.tensor([input_ids]),
                    token_type_ids=torch.tensor([segment_ids]),
                    return_dict=True) 

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # ======== Reconstruct Answer ========
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer = tokens[answer_start]

    for i in range(answer_start + 1, answer_end + 1):
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')

In [ ]:
transformers.logging.set_verbosity_error()

In [ ]:
def narrow_to_topk(topk):
    num_queries = 27
    tfidf = {}
    with open('sample_test_tokenize2.tfidf.tfidf.output', 'r') as data:
        for line in data:
            queryid, docid = line.split()
            if queryid not in tfidf:
                tfidf[queryid] = list()
            tfidf[queryid].append(docid)

    reljudge = {}
    with open('answers.txt', 'r') as data:
        for line in data:
            queryid, docid = line.split()
            if queryid not in reljudge:
                reljudge[queryid] = set()
            reljudge[queryid].add(docid)

    total_retrieved_unique = set()
    for queryid in range(1, num_queries+1):
        qid = str(queryid)
        top_retrieved = set()
        for i in range(topk):
            if queryid <= 27 and i < len(tfidf[qid]): 
                top_retrieved.add(tfidf[qid][i])
                total_retrieved_unique.add(tfidf[qid][i])
        rel_returned = top_retrieved.intersection(reljudge[qid])
    retrieved_list = list(total_retrieved_unique)

    narrowed_passages = []
    folder_addr = "sample_train_set"
    queries = "queries.txt"

    folder_path = os.path.join(current_path, folder_addr)
    files = os.listdir(folder_path)
    for file_name in retrieved_list:
        file_name = file_name + '.json'
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as read_file:
            data = json.load(read_file)
            txt = ""
            id = data['paper_id']
            for each in data['body_text']:
                txt += each['text']
            narrowed_passages.append((id, txt))
    return narrowed_passages


In [ ]:
narrowed_passage = narrow_to_topk(50)
len(narrowed_passage)

274

In [ ]:
query = "What are the risks of COVID-19 infection in pregnant women?"
#query = "What else does the bassist for Death From Above play?"
#query = "What projects is Jesse Keeler involved in?"

segmented_docs = segment_documents(narrowed_passage, 450)
candidate_docs = get_top_k_articles(query, segmented_docs, 3)

for i in candidate_docs:
    txt = i[1]
    id = i[0]
    answer_question(query, txt)
    print ("Reference Document: ", id)

Answer: "[SEP]"
Reference Document:  9fa0388a057382f207419559b0b5a791f66391b3
Answer: "[CLS]"
Reference Document:  9fa0388a057382f207419559b0b5a791f66391b3
Answer: "predisposes women to respiratory complications of viral infection"
Reference Document:  60ef5f349a591456899d5d7cb77c965d46b63260


In [ ]:
def test_bert(top_k):
    query_path = os.path.join("queries.txt")
    query_content = open(query_path, 'r')
    query_lines = query_content.readlines()
    list_similarity_scores = []
    final_output = []
    for query in query_lines:
        query_id = query.split(' ')[0]
        query = query[2:]
        candidate_docs = get_top_k_articles(query, segmented_docs, top_k)
        retrieved_list = []
        for candidate in candidate_docs:
            retrieved_list.append(candidate[0])
        for file_id in retrieved_list:
            output = str(query_id) + ' ' + file_id
            final_output.append(output)
    
    output_file = f'sample_test_bert.output'
    with open(output_file, 'w') as output:
        for line in final_output:
            line = line + '\n'
            output.write(line)

In [ ]:
test_bert(10)

In [ ]:
print('BERT')
marco_avg('sample_test_bert.output')

BERT
Top 1
  retrieved(relevant) = 3
  retrieved(total) = 27
  precision(macro-avg) = 11.11%
  recall(macro-avg) = 3.00%
total 18
Top 5
  retrieved(relevant) = 9
  retrieved(total) = 135
  precision(macro-avg) = 6.67%
  recall(macro-avg) = 9.67%
total 50
Top 10
  retrieved(relevant) = 11
  retrieved(total) = 270
  precision(macro-avg) = 4.07%
  recall(macro-avg) = 10.35%
total 78
Top 50
  retrieved(relevant) = 11
  retrieved(total) = 1350
  precision(macro-avg) = 0.81%
  recall(macro-avg) = 10.35%
total 78
Top 100
  retrieved(relevant) = 11
  retrieved(total) = 2700
  precision(macro-avg) = 0.41%
  recall(macro-avg) = 10.35%
total 78
Top 500
  retrieved(relevant) = 11
  retrieved(total) = 13500
  precision(macro-avg) = 0.08%
  recall(macro-avg) = 10.35%
total 78
Top 1000
  retrieved(relevant) = 11
  retrieved(total) = 27000
  precision(macro-avg) = 0.04%
  recall(macro-avg) = 10.35%
total 78


In [ ]:
def QA(query):
    answers_id = get_top_k_articles(query, segmented_docs, 5)
    count = 1
    for id, txt in answers_id:
        print("========================================")
        if len(metadata[metadata['sha'] == id]) != 0:
            print(count, id)
            print("Title: ", metadata[metadata['sha'] == id]['title'].item())
            print("Abstract: ", metadata[metadata['sha'] == id]['abstract'].item())
        else:
            print(count, id)
            print("Info not found")
        count += 1

# User Interface

In [ ]:
query_from_user = input("Enter a query related to COVID-19: ")
QA(query_from_user)

Enter a query related to COVID-19: what are the symptoms?
1 2f5e6026c5f4241d0636dfd05e0c230f65d58021
Title:  Persistent viral shedding of SARS-CoV-2 in faeces - a rapid review
Abstract:  Background and aims In addition to respiratory symptoms, patients with COVID-19 can present with gastrointestinal complaints suggesting a possible faeco-oral transmission route. The primary aim of this review is to establish the incidence and timing of positive faecal samples for the SARS-CoV-2 virus in patients with COVID-19. Methods A systematic literature review was performed to identify studies describing COVID-19 patients tested for the virus in their stool. Data were extracted concerning the nature of the test, number and timing of positive samples, incidence of positive faecal tests after negative nasopharyngeal swabs and any evidence of viable faecal virus or faeco-oral transmission of the virus. Results There were 26 relevant articles identified. Combining these results demonstrated that 53.9%